In [12]:
import pandas as pd
import numpy as np
import nltk
import pyspark
import matplotlib.pyplot as plt
import os.path as osp
import os
import shutil
from open_secrets_schema import Schema
import sys
from IPython.display import display, HTML

%matplotlib notebook

get legislator information - THOMAS, Bioguide, CRP/OpenSecrets, GovTrack ID, plus names etc. from the [@unitedstates](https://github.com/unitedstates) GitHub page

In [2]:
!whoami

matt


In [3]:
!pwd

/home/matt/etl


In [4]:
!ls

bill-metadata			 open-secrets.ipynb
bill-text			 open_secrets_schema.py
bill-text.ipynb			 propublica-bill-meta.ipynb
legislators			 __pycache__
open-secrets			 unitedstates-legislators.ipynb
open-secrets-expenditures.ipynb  views


In [5]:
!mkdir -p legislators
os.chdir('legislators')

In [6]:
legiscur = r'https://theunitedstates.io/congress-legislators/legislators-current.csv'
legisprev = r'https://theunitedstates.io/congress-legislators/legislators-historical.csv'

In [8]:
for legisname, legisurl in zip(['legiscur', 'legisprev'], [legiscur, legisprev]):
    display(HTML(f'<h2>{legisname}</h2>'))
    cmd = f'wget -O {legisname}.csv {legisurl}'
    print(f'Executing command: {cmd}')
    !$cmd
    display(HTML('<hr>'))

Executing command: wget -O legiscur.csv https://theunitedstates.io/congress-legislators/legislators-current.csv
--2018-10-20 23:39:15--  https://theunitedstates.io/congress-legislators/legislators-current.csv
Resolving theunitedstates.io (theunitedstates.io)... 104.27.152.137, 104.27.153.137, 2606:4700:30::681b:9889, ...
Connecting to theunitedstates.io (theunitedstates.io)|104.27.152.137|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘legiscur.csv’

    [ <=>                                   ] 191,762     --.-K/s   in 0.003s  

2018-10-20 23:39:15 (68.4 MB/s) - ‘legiscur.csv’ saved [191762]



Executing command: wget -O legisprev.csv https://theunitedstates.io/congress-legislators/legislators-historical.csv
--2018-10-20 23:39:15--  https://theunitedstates.io/congress-legislators/legislators-historical.csv
Resolving theunitedstates.io (theunitedstates.io)... 104.27.152.137, 104.27.153.137, 2606:4700:30::681b:9989, ...
Connecting to theunitedstates.io (theunitedstates.io)|104.27.152.137|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘legisprev.csv’

    [ <=>                                   ] 1,404,675   --.-K/s   in 0.01s   

2018-10-20 23:39:15 (119 MB/s) - ‘legisprev.csv’ saved [1404675]



In [9]:
!ls

legiscur.csv  legisprev.csv


In [22]:
lcdf = pd.read_csv('legiscur.csv')
lcdf.head(3)

,last_name,first_name,middle_name,suffix,nickname,full_name,birthday,gender,type,state,...,opensecrets_id,lis_id,fec_ids,cspan_id,govtrack_id,votesmart_id,ballotpedia_id,washington_post_id,icpsr_id,wikipedia_id
0,Brown,Sherrod,NaN,NaN,NaN,Sherrod Brown,1952-11-09,M,sen,OH,...,N00003535,S307,"H2OH13033,S6OH00163",5051.0,400050,27018.0,Sherrod Brown,NaN,29389.0,Sherrod Brown
1,Cantwell,Maria,NaN,NaN,NaN,Maria Cantwell,1958-10-13,F,sen,WA,...,N00007836,S275,"S8WA00194,H2WA01054",26137.0,300018,27122.0,Maria Cantwell,NaN,39310.0,Maria Cantwell
2,Cardin,Benjamin,L.,NaN,NaN,Benjamin L. Cardin,1943-10-05,M,sen,MD,...,N00001955,S308,"H6MD03177,S6MD03177",4004.0,400064,26888.0,Ben Cardin,NaN,15408.0,Ben Cardin


In [23]:
lpdf = pd.read_csv('legisprev.csv')
lpdf.head(3)

,last_name,first_name,middle_name,suffix,nickname,full_name,birthday,gender,type,state,...,opensecrets_id,lis_id,fec_ids,cspan_id,govtrack_id,votesmart_id,ballotpedia_id,washington_post_id,icpsr_id,wikipedia_id
0,Bassett,Richard,NaN,NaN,NaN,NaN,1745-04-02,M,sen,DE,...,NaN,NaN,NaN,NaN,401222,NaN,NaN,NaN,507.0,Richard Bassett (politician)
1,Bland,Theodorick,NaN,NaN,NaN,NaN,1742-03-21,M,rep,VA,...,NaN,NaN,NaN,NaN,401521,NaN,NaN,NaN,786.0,Theodorick Bland (congressman)
2,Burke,Aedanus,NaN,NaN,NaN,NaN,1743-06-16,M,rep,SC,...,NaN,NaN,NaN,NaN,402032,NaN,NaN,NaN,1260.0,Aedanus Burke


Same columns?

In [24]:
def check_schemas_equal(df1, df2):
    bad = False
    for c in df1.columns:
        if c not in df2.columns:
            print(f'column {c} in df1, not in df2!', file=sys.stderr)
            bad = True
        elif df1[c].dtype != df2[c].dtype:
            print(f'column {c} dtype={df1[c].dtype} in df1, dtype={df2[c].dtype} in df2!', file=sys.stderr)
            bad = True
    for c in df2.columns:
        if c not in df1.columns:
            print(f'column {c} in df2, not in df1!', file=sys.stderr)
            bad = True
        elif df1[c].dtype != df2[c].dtype:
            print(f'column {c} dtype={df2[c].dtype} in df2, dtype={df1[c].dtype} in df1!', file=sys.stderr)
            bad = True
    if bad:
        print(f'schemas are not equal!')
    else:
        print('everything looks good')

In [25]:
check_schemas_equal(lcdf, lpdf)

schemas are not equal!


column twitter dtype=object in df1, dtype=float64 in df2!
column facebook dtype=object in df1, dtype=float64 in df2!
column youtube dtype=object in df1, dtype=float64 in df2!
column youtube_id dtype=object in df1, dtype=float64 in df2!
column twitter dtype=float64 in df2, dtype=object in df1!
column facebook dtype=float64 in df2, dtype=object in df1!
column youtube dtype=float64 in df2, dtype=object in df1!
column youtube_id dtype=float64 in df2, dtype=object in df1!


Social media is probably all nulls in the `legislators-historical` data - let's verify this

In [26]:
for col in ['twitter', 'facebook', 'youtube', 'youtube_id']:
    if lpdf[col].isnull().sum() == len(lpdf[col]):
        print(f'{col} is all nulls in lpdf')

twitter is all nulls in lpdf
facebook is all nulls in lpdf
youtube is all nulls in lpdf
youtube_id is all nulls in lpdf


Fix by reading in as object

In [27]:
lpdf = pd.read_csv('legisprev.csv', dtype={c: 'str' for c in ['twitter', 'facebook', 'youtube', 'youtube_id']})

verify that everything looks good

In [28]:
check_schemas_equal(lcdf, lpdf)

everything looks good


Now that schemas match, we can aggregate the DataFrames together and save to Hive

In [29]:
fulldf = lpdf.append(lcdf)

In [30]:
assert len(fulldf) == len(lpdf) + len(lcdf)

In [31]:
fulldf

,last_name,first_name,middle_name,suffix,nickname,full_name,birthday,gender,type,state,...,opensecrets_id,lis_id,fec_ids,cspan_id,govtrack_id,votesmart_id,ballotpedia_id,washington_post_id,icpsr_id,wikipedia_id
0,Bassett,Richard,NaN,NaN,NaN,NaN,1745-04-02,M,sen,DE,...,NaN,NaN,NaN,NaN,401222,NaN,NaN,NaN,507.0,Richard Bassett (politician)
1,Bland,Theodorick,NaN,NaN,NaN,NaN,1742-03-21,M,rep,VA,...,NaN,NaN,NaN,NaN,401521,NaN,NaN,NaN,786.0,Theodorick Bland (congressman)
2,Burke,Aedanus,NaN,NaN,NaN,NaN,1743-06-16,M,rep,SC,...,NaN,NaN,NaN,NaN,402032,NaN,NaN,NaN,1260.0,Aedanus Burke
3,Carroll,Daniel,NaN,NaN,NaN,NaN,1730-07-22,M,rep,MD,...,NaN,NaN,NaN,NaN,402334,NaN,NaN,NaN,1538.0,Daniel Carroll
4,Clymer,George,NaN,NaN,NaN,NaN,1739-03-16,M,rep,PA,...,NaN,NaN,NaN,NaN,402671,NaN,NaN,NaN,1859.0,George Clymer
5,Contee,Benjamin,NaN,NaN,NaN,NaN,NaN,M,rep,MD,...,NaN,NaN,NaN,NaN,402834,NaN,NaN,NaN,2010.0,Benjamin Contee
6,Dalton,Tristram,NaN,NaN,NaN,NaN,1738-05-28,M,sen,MA,...,NaN,NaN,NaN,NaN,403156,NaN,NaN,NaN,2307.0,Tristram Dalton
7,Elmer,Jonathan,NaN,NaN,NaN,NaN,1745-11-29,M,sen,NJ,...,NaN,NaN,NaN,NaN,403846,NaN,NaN,NaN,2943.0,Jonathan Elmer
8,Few,William,NaN,NaN,NaN,NaN,1748-06-08,M,sen,GA,...,NaN,NaN,NaN,NaN,404057,NaN,NaN,NaN,3128.0,William Few
9,Floyd,William,NaN,NaN,NaN,NaN,1734-12-17,M,rep,NY,...,NaN,NaN,NaN,NaN,404179,NaN,NaN,NaN,3237.0,William Floyd


Our numeric ID columns have been cast to float... of course.

In [33]:
for c in fulldf:
    display(HTML(f'<h2>{c}</h2>'))
    print(fulldf[c].head(5))
    print('--------')
    print(fulldf[c].tail(5))
    display(HTML('<hr>'))

0    Bassett
1      Bland
2      Burke
3    Carroll
4     Clymer
Name: last_name, dtype: object
--------
529         Lamb
530        Lesko
531        Cloud
532          Kyl
533    Balderson
Name: last_name, dtype: object


0       Richard
1    Theodorick
2       Aedanus
3        Daniel
4        George
Name: first_name, dtype: object
--------
529      Conor
530     Debbie
531    Michael
532        Jon
533       Troy
Name: first_name, dtype: object


0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: middle_name, dtype: object
--------
529    NaN
530    NaN
531    NaN
532    NaN
533    NaN
Name: middle_name, dtype: object


0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: suffix, dtype: object
--------
529    NaN
530    NaN
531    NaN
532    NaN
533    NaN
Name: suffix, dtype: object


0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: nickname, dtype: object
--------
529    NaN
530    NaN
531    NaN
532    NaN
533    NaN
Name: nickname, dtype: object


...

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: votesmart_id, dtype: float64
--------
529    177656.0
530    106483.0
531    177350.0
532     26721.0
533         NaN
Name: votesmart_id, dtype: float64


0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: ballotpedia_id, dtype: object
--------
529               Conor Lamb
530             Debbie Lesko
531    Michael Cloud (Texas)
532                      NaN
533           Troy Balderson
Name: ballotpedia_id, dtype: object


0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: washington_post_id, dtype: float64
--------
529   NaN
530   NaN
531   NaN
532   NaN
533   NaN
Name: washington_post_id, dtype: float64


0     507.0
1     786.0
2    1260.0
3    1538.0
4    1859.0
Name: icpsr_id, dtype: float64
--------
529        NaN
530        NaN
531        NaN
532    15429.0
533        NaN
Name: icpsr_id, dtype: float64


0      Richard Bassett (politician)
1    Theodorick Bland (congressman)
2                     Aedanus Burke
3                    Daniel Carroll
4                     George Clymer
Name: wikipedia_id, dtype: object
--------
529        Conor Lamb
530      Debbie Lesko
531     Michael Cloud
532           Jon Kyl
533    Troy Balderson
Name: wikipedia_id, dtype: object


With the exception of `birthday`, everything can probably be read in as a string anyway. Interesting thing to note though is that the `FEC` column is `fec_ids`, plural.

In [38]:
lpdf = pd.read_csv('legisprev.csv', dtype='str')

In [39]:
lcdf = pd.read_csv('legiscur.csv', dtype='str')

In [40]:
check_schemas_equal(lpdf, lcdf)

everything looks good


In [41]:
fulldf = lpdf.append(lcdf)
fulldf

,last_name,first_name,middle_name,suffix,nickname,full_name,birthday,gender,type,state,...,opensecrets_id,lis_id,fec_ids,cspan_id,govtrack_id,votesmart_id,ballotpedia_id,washington_post_id,icpsr_id,wikipedia_id
0,Bassett,Richard,NaN,NaN,NaN,NaN,1745-04-02,M,sen,DE,...,NaN,NaN,NaN,NaN,401222,NaN,NaN,NaN,507,Richard Bassett (politician)
1,Bland,Theodorick,NaN,NaN,NaN,NaN,1742-03-21,M,rep,VA,...,NaN,NaN,NaN,NaN,401521,NaN,NaN,NaN,786,Theodorick Bland (congressman)
2,Burke,Aedanus,NaN,NaN,NaN,NaN,1743-06-16,M,rep,SC,...,NaN,NaN,NaN,NaN,402032,NaN,NaN,NaN,1260,Aedanus Burke
3,Carroll,Daniel,NaN,NaN,NaN,NaN,1730-07-22,M,rep,MD,...,NaN,NaN,NaN,NaN,402334,NaN,NaN,NaN,1538,Daniel Carroll
4,Clymer,George,NaN,NaN,NaN,NaN,1739-03-16,M,rep,PA,...,NaN,NaN,NaN,NaN,402671,NaN,NaN,NaN,1859,George Clymer
5,Contee,Benjamin,NaN,NaN,NaN,NaN,NaN,M,rep,MD,...,NaN,NaN,NaN,NaN,402834,NaN,NaN,NaN,2010,Benjamin Contee
6,Dalton,Tristram,NaN,NaN,NaN,NaN,1738-05-28,M,sen,MA,...,NaN,NaN,NaN,NaN,403156,NaN,NaN,NaN,2307,Tristram Dalton
7,Elmer,Jonathan,NaN,NaN,NaN,NaN,1745-11-29,M,sen,NJ,...,NaN,NaN,NaN,NaN,403846,NaN,NaN,NaN,2943,Jonathan Elmer
8,Few,William,NaN,NaN,NaN,NaN,1748-06-08,M,sen,GA,...,NaN,NaN,NaN,NaN,404057,NaN,NaN,NaN,3128,William Few
9,Floyd,William,NaN,NaN,NaN,NaN,1734-12-17,M,rep,NY,...,NaN,NaN,NaN,NaN,404179,NaN,NaN,NaN,3237,William Floyd


ID columns look good. Now, how about that `fec_ids` column?

In [42]:
fulldf['fec_ids'].value_counts()

H6FL09179              1
H0AR03055              1
H6NV03139              1
S6UT00063,P00003806    1
H4MA03022              1
H2AL01077              1
H2IL20042              1
H2NY03089              1
H8NY23084              1
H0CA11022              1
S8MI00158              1
H6OR03064              1
H2CA30143              1
H2NJ13075,S6NJ00289    1
H8NJ05052              1
H4IL12060              1
H0AR01083              1
H6MN02149              1
H4AL01123              1
H0MO02148              1
H2CA20094              1
H8CT04057              1
H4AZ04016              1
H6WA03135              1
H6CA05195              1
S2ME00158,H4ME02234    1
H4LA05130              1
H0MO00019              1
H8WY00148              1
H6KS01146              1
                      ..
H4KY02030              1
S0IN00053              1
H0VA01052              1
H8KS02090              1
H8AZ01104              1
H8CA34068              1
H8NV01071              1
H6NH01230              1
H2TX31044              1


In [48]:
fulldf.fec_ids.str.split(',').map(lambda x: 0 if type(x) != list else len(x)).max()

3

I think this calls for two tables: one with candidate data and one with FEC data (keyed on Bioguide). Let's make sure that Bioguide can serve as a suitable PK.

In [49]:
fulldf.bioguide_id.value_counts().max()

1

In [51]:
fulldf.bioguide_id.isnull().sum()

0

No nulls, no repetitions, great.

In [66]:
def bgf(df):
    df = df.dropna(subset=['fec_ids'])
    for _, r in df.iterrows():
        for fec_id in r['fec_ids'].split(','):
            yield r['bioguide_id'], fec_id

In [72]:
bioguide_fec_table = pd.DataFrame(columns=['bioguide_id', 'fec_id'],
                                  data=bgf(fulldf))

In [74]:
fulldf = fulldf.drop('fec_ids', axis=1)

In [93]:
bioguide_fec_table.to_csv('bioguide_fec_table.csv',
                          index=False,
                          header=False)

In [94]:
fulldf.to_csv('legislators.csv',
              index=False,
              header=False)

## Save to Hive

In [78]:
sc = pyspark.sql.SparkSession.Builder()\
            .master('yarn')\
            .config('spark.num.executors', '1')\
            .config('spark.executor.cores', '2')\
            .config('spark.executor.memory', '2g')\
            .appName('load-legislators')\
            .enableHiveSupport()\
            .getOrCreate()

In [109]:
def etl_text_file(text_file_path, schema_str, table_name, data_source_name, column_datetime_formats=None,
                  csv_quotechar='|', csv_delimiter=','):
    schema = Schema(schema_str)
    schema.register_column_datetime_formats(column_datetime_formats)
    textFileName = osp.split(text_file_path)[1]
    stagingFolder = f'/staging/{table_name}'
    !hdfs dfs -rm -r -f {stagingFolder}/{textFileName}
    !hdfs dfs -mkdir -p {stagingFolder}
    !hdfs dfs -put {text_file_path} {stagingFolder}/{textFileName}
    !hdfs dfs -chmod -R 777 {stagingFolder}
    with open(f'{table_name}.sql', 'w') as f:
        f.write(schema.hive_ddl(tableName=table_name, externalTableLocation=stagingFolder,
                                csv_quotechar=csv_quotechar, csv_delimiter=csv_delimiter))
    !/opt/anaconda3/bin/beeline -u jdbc:hive2://cap1:10000 -n matt -p anything -f {table_name}.sql
    
    # Convert to the proper schema and write as parquet
    finalTableName = f'pq_{data_source_name}_{table_name}'
    with open(f'{finalTableName}.sql', 'w') as f:
        f.write(schema.hive_ddl_parquet_alter(tableName=finalTableName, originalTableName=table_name))
    
    dfWithSchema = sc.sql(schema.hive_ddl_parquet_select(tableName=finalTableName, originalTableName=table_name))
    sc.sql(f'DROP TABLE IF EXISTS {finalTableName}')
    dfWithSchema.write.saveAsTable(finalTableName)
    !/opt/anaconda3/bin/beeline -u jdbc:hive2://cap1:10000 -n matt -p anything -f {finalTableName}.sql
    
    # clean up -- drop the intermediate (text file) table and delete staged data from HDFS
    sc.sql(f'DROP TABLE {table_name}')
    !hdfs dfs -rm -r -f {stagingFolder}
    
    print('done')

In [88]:
pd.options.display.max_columns = 500

In [90]:
fulldf.head(3)

,last_name,first_name,middle_name,suffix,nickname,full_name,birthday,gender,type,state,district,senate_class,party,url,address,phone,contact_form,rss_url,twitter,facebook,youtube,youtube_id,bioguide_id,thomas_id,opensecrets_id,lis_id,cspan_id,govtrack_id,votesmart_id,ballotpedia_id,washington_post_id,icpsr_id,wikipedia_id
0,Bassett,Richard,NaN,NaN,NaN,NaN,1745-04-02,M,sen,DE,NaN,2,Anti-Administration,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B000226,NaN,NaN,NaN,NaN,401222,NaN,NaN,NaN,507,Richard Bassett (politician)
1,Bland,Theodorick,NaN,NaN,NaN,NaN,1742-03-21,M,rep,VA,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B000546,NaN,NaN,NaN,NaN,401521,NaN,NaN,NaN,786,Theodorick Bland (congressman)
2,Burke,Aedanus,NaN,NaN,NaN,NaN,1743-06-16,M,rep,SC,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B001086,NaN,NaN,NaN,NaN,402032,NaN,NaN,NaN,1260,Aedanus Burke


## Load `legislators`

`legislatorsSchemaStr` from: https://github.com/unitedstates/congress-legislators

In [97]:
legislatorsSchemaStr = ["""last_name
The legislator's last name. Many last names include non-ASCII characters. When building search systems, it is advised to index both the raw value as well as a value with extended characters replaced with their ASCII equivalents (in Python that's: u"".join(c for c in unicodedata.normalize('NFKD', lastname) if not unicodedata.combining(c))).
Text (30)
US""",
                        
"""first_name
The legislator's first name. Sometimes a first initial and period (e.g. in W. Todd Akin), in which case it is suggested to not use the first name for display purposes.
Text (30)
US""",
                        
"""middle_name
The legislator's middle name or middle initial (with period).
Text (30)
US""",

"""suffix
A suffix on the legislator's name, such as "Jr." or "III".
Text (12)
US""",
                        
"""nickname
The legislator's nick name when used as a common alternative to his first name.
Text (12)
US""",

"""full_name
The full name of the legislator according to the House or Senate (usually first, middle initial, nickname, last, and suffix). Present for those serving on 2012-10-30 and later.
Text (12)
US""",

"""birthday
The legislator's birthday, in YYYY-MM-DD format.
yyyy-MM-dd
US""",

"""gender
The legislator's gender, either "M" or "F". (In historical data, we've worked backwards from history.house.gov's Women in Congress feature.
Text (1)
US""",

"""type
The type of the term. Either "sen" for senators or "rep" for representatives and delegates to the House.
Text (3)
US""",

"""state
The two-letter, uppercase USPS abbreviation for the state that the legislator is serving from. See below.
Text (2)
US""",

"""district
For representatives, the district number they are serving from. At-large districts are district 0. In historical data, unknown district numbers are recorded as -1.
Text (8)
US""",

"""senate_class
Senator election year class. Class 1 is 2 years before presidential election year, class 2 is presidential election year, class 3 is 2 years after presidential election year.
Text(1)
US""",

"""party
The political party of the legislator. If the legislator changed parties, this is the most recent party held during the term and party_affiliations will be set. Values are typically "Democrat", "Independent", or "Republican". The value typically matches the political party of the legislator on the ballot in his or her last election, although for state affiliate parties such as "Democratic Farmer Labor" we will use the national party name ("Democrat") instead to keep the values of this field normalized.
Text (20)
US""",

"""url
The official website URL of the legislator (only valid if the term is current).
Text (50)
US""",

"""address
The mailing address of the legislator's Washington, D.C. office (only valid if the term is current, otherwise the last known address).
Text (80)
US""",

"""phone
The phone number of the legislator's Washington, D.C. office (only valid if the term is current, otherwise the last known number).
Text (20)
US""",

"""contact_form
The website URL of the contact page on the legislator's official website (only valid if the term is current, otherwise the last known URL).
Text (80)
US""",

"""rss_url
The URL to the official website's RSS feed (only valid if the term is current, otherwise the last known URL).
Text (80)
US""",

"""twitter
The current official Twitter handle of the legislator.
Text (80)
US""",

"""facebook
The username of the current official Facebook presence of the legislator.
Text (80)
US""",

"""youtube
The current official YouTube username of the legislator.
Text (80)
US""",

"""youtube_id
The current official YouTube channel ID of the legislator.
Text (80)
US""",

"""bioguide_id
The alphanumeric ID for this legislator in http://bioguide.congress.gov. Note that at one time some legislators (women who had changed their name when they got married) had two entries on the bioguide website. Only one bioguide ID is included here. This is the best field to use as a primary key.
Text (15)
US""",

"""thomas_id
The numeric ID for this legislator on http://thomas.gov and http://beta.congress.gov. The ID is stored as a string with leading zeros preserved.
Text (12)
US""",

"""opensecrets_id
The alphanumeric ID for this legislator on OpenSecrets.org.
Text (15)
US""",

"""lis_id
The alphanumeric ID for this legislator found in Senate roll call votes (http://www.senate.gov/pagelayout/legislative/a_three_sections_with_teasers/votes.htm).
Text (15)
US""",

"""cspan_id
The numeric ID for this legislator on C-SPAN's video website, e.g. http://www.c-spanvideo.org/person/1745 (stored as an integer).
Text (15)
US""",

"""govtrack_id
The numeric ID for this legislator on GovTrack.us (stored as an integer).
Text (15)
US""",

"""votesmart_id
The numeric ID for this legislator on VoteSmart.org (stored as an integer).
Text (15)
US""",

"""ballotpedia_id
The ballotpedia.org page name for the person (spaces are given as spaces, not underscores).
Text (15)
US""",

"""washington_post_id
<<This field is not described in the source data dictionary - MTP 10/21/2018>>
Text (15)
US""",

"""icpsr_id
The numeric ID for this legislator in Keith Poole's VoteView.com website, originally based on an ID system by the Interuniversity Consortium for Political and Social Research (stored as an integer).
Text (15)
US""",

"""wikipedia_id
The Wikipedia page name for the person (spaces are given as spaces, not underscores).
Text (15)
US"""
]

In [113]:
from importlib import reload
import open_secrets_schema
reload(open_secrets_schema)
from open_secrets_schema import Schema

In [114]:
etl_text_file('legislators.csv',
              legislatorsSchemaStr,
              'legislators',
              'us',
              column_datetime_formats={'birthday': 'yyyy-MM-dd'},
              csv_quotechar='\\"',
              csv_delimiter=',')

2018-10-21 10:42:30,894 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2018-10-21 10:42:32,953 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2018-10-21 10:42:35,012 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2018-10-21 10:42:37,283 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Connecting to jdbc:hive2://cap1:10000
2018-10-21 10:42:39 INFO  Utils:310 - Supplied authorities: cap1:10000
2018-10-21 10:42:39 INFO  Utils:397 - Resolved authority: cap1:10000
2018-10-21 10:42:39 INFO  HiveConnection:203 - Will try to open client transport with JDBC Uri: jdbc:hive2://cap1:10000
Connected to: Spark SQL (version 2.3.2)
Driver: Hive JDBC (version 1.2.1.spark2)
Trans

+---------+--+
| Result  |
+---------+--+
+---------+--+
No rows selected (0.103 seconds)
g-dtype=Text (80), source=US';nt official Twitter handle of the legislator., ori 
+---------+--+
| Result  |
+---------+--+
+---------+--+
No rows selected (0.109 seconds)
 the legislator., orig-dtype=Text (80), source=US';official Facebook presence of 
+---------+--+
| Result  |
+---------+--+
+---------+--+
No rows selected (0.104 seconds)
rig-dtype=Text (80), source=US'; official YouTube username of the legislator., o 
+---------+--+
| Result  |
+---------+--+
+---------+--+
No rows selected (0.103 seconds)
ator., orig-dtype=Text (80), source=US';fficial YouTube channel ID of the legisl 
+---------+--+
| Result  |
+---------+--+
+---------+--+
No rows selected (0.101 seconds)
 key., orig-dtype=Text (15), source=US';is is the best field to use as a primary 
+---------+--+
| Result  |
+---------+--+
+---------+--+
No rows selected (0.105 seconds)
reserved., orig-dtype=Text (12), source=US';red as

Verify by selecting some rows and describing the table:

In [115]:
!beeline -u jdbc:hive2://cap1:10000 -n matt -p anything -e 'SELECT * FROM pq_us_legislators LIMIT 10;'
!beeline -u jdbc:hive2://cap1:10000 -n matt -p anything -e 'DESCRIBE pq_us_legislators;'

Connecting to jdbc:hive2://cap1:10000
2018-10-21 10:43:55 INFO  Utils:310 - Supplied authorities: cap1:10000
2018-10-21 10:43:55 INFO  Utils:397 - Resolved authority: cap1:10000
2018-10-21 10:43:55 INFO  HiveConnection:203 - Will try to open client transport with JDBC Uri: jdbc:hive2://cap1:10000
Connected to: Spark SQL (version 2.3.2)
Driver: Hive JDBC (version 1.2.1.spark2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
+------------+-------------+--------------+---------+-----------+------------+-------------+---------+-------+--------+-----------+---------------+-------------+------+----------+--------+---------------+----------+----------+-----------+----------+-------------+--------------+------------+-----------------+---------+-----------+--------------+---------------+-----------------+---------------------+-----------+-----------------------+--+
| last_name  | first_name  | middle_name  | suffix  | nickname  | full_name  |  birthday   | gender  | type  | state  | district

Beeline version 1.2.1.spark2 by Apache Hive
Closing: 0: jdbc:hive2://cap1:10000


## Load `fec_ids`

`bgfSchemaStr` from: https://github.com/unitedstates/congress-legislators

In [116]:
bgfSchemaStr = ["""bioguide_id
The alphanumeric ID for this legislator in http://bioguide.congress.gov. Note that at one time some legislators (women who had changed their name when they got married) had two entries on the bioguide website. Only one bioguide ID is included here. This is the best field to use as a primary key.
Text (15)
US""",
                        
"""fec_id
A list of IDs for this legislator in Federal Election Commission data. In the CSV format, the fec_ids column is comma-separated. <<Post-processed to split out to one row per ID - MTP 10/21/2018>>
Text (15)
US"""
]

In [117]:
etl_text_file('bioguide_fec_table.csv',
              bgfSchemaStr,
              'fec_ids',
              'us',
              csv_quotechar='\\"',
              csv_delimiter=',')

2018-10-21 10:44:14,189 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2018-10-21 10:44:16,275 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2018-10-21 10:44:18,373 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2018-10-21 10:44:20,596 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Connecting to jdbc:hive2://cap1:10000
2018-10-21 10:44:22 INFO  Utils:310 - Supplied authorities: cap1:10000
2018-10-21 10:44:22 INFO  Utils:397 - Resolved authority: cap1:10000
2018-10-21 10:44:22 INFO  HiveConnection:203 - Will try to open client transport with JDBC Uri: jdbc:hive2://cap1:10000
Connected to: Spark SQL (version 2.3.2)
Driver: Hive JDBC (version 1.2.1.spark2)
Trans

Again, verify by selecting rows and showing schema:

In [118]:
!beeline -u jdbc:hive2://cap1:10000 -n matt -p anything -e 'SELECT * FROM pq_us_fec_ids LIMIT 10;'
!beeline -u jdbc:hive2://cap1:10000 -n matt -p anything -e 'DESCRIBE pq_us_fec_ids;'

Connecting to jdbc:hive2://cap1:10000
2018-10-21 10:45:33 INFO  Utils:310 - Supplied authorities: cap1:10000
2018-10-21 10:45:33 INFO  Utils:397 - Resolved authority: cap1:10000
2018-10-21 10:45:33 INFO  HiveConnection:203 - Will try to open client transport with JDBC Uri: jdbc:hive2://cap1:10000
Connected to: Spark SQL (version 2.3.2)
Driver: Hive JDBC (version 1.2.1.spark2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
+--------------+------------+--+
| bioguide_id  |   fec_id   |
+--------------+------------+--+
| B001236      | H2AR03176  |
| B001236      | S0AR00150  |
| B001245      | H2GU00033  |
| B000755      | H6TX08100  |
| B001227      | H8PA01153  |
| B001274      | H0AL05163  |
| B001260      | H6FL13148  |
| B001275      | H0IN08114  |
| B001248      | H2TX26093  |
| B001135      | S4NC00089  |
+--------------+------------+--+
10 rows selected (0.203 seconds)
Beeline version 1.2.1.spark2 by Apache Hive
Closing: 0: jdbc:hive2://cap1:10000
Connecting to jdbc:hive2://c